In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import bilby
import os
import requests # used to download O1 and O2 data
import corner

import sys
sys.path.insert(0,'../src')
from util import *
from cross_correlation import *
from postprocessing import *
from pe import *
from constants import *

## Get O1 and O2 data

In [ ]:
# taken from here: https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests/16696317#16696317
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename

### Get O1 data and plot it as a santy check

In [ ]:
try:
    fname_O1='data/figure1.dat'
    freqs_1,Y_f_1,two_sigma_f_1=np.loadtxt(fname_O1,unpack=True,skiprows=1)
except OSError:
    fname_O1=download_file('https://dcc.ligo.org/public/0139/G1602446/003/figure1.dat')
    make_dir('data')
    fname_O1_new = 'data/{fname_O1}'.format(fname_O1=fname_O1)
    os.rename(fname_O1,fname_O1_new)
    freqs_1,Y_f_1,two_sigma_f_1=np.loadtxt(fname_O1_new,unpack=True,skiprows=1)
    
sigma_f_1=two_sigma_f_1/2
var_f_1=sigma_f_1**2

HubbleConstantCorrection = 0.68**2 / 0.679**2 # H0 choice changed from O1 to O2
Y_f_1 = Y_f_1 * HubbleConstantCorrection
sigma_f_1 = sigma_f_1 * HubbleConstantCorrection
var_f_1 = var_f_1 * HubbleConstantCorrection**2

In [ ]:
plt.plot(freqs_1,Y_f_1,color='blue',label='Y(f)')
plt.plot(freqs_1,sigma_f_1,color='black',label='+/-sigma(f)')
plt.plot(freqs_1,-sigma_f_1,color='black')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Omega')
plt.xlim(20,60)
plt.ylim(-1e-5,1e-5)
plt.legend()


### Get O2 data and plot it as a santy check

In [ ]:
try:
    fname_O2='data/Figure_1_Cf_spectra_O2.dat'
    freqs_2,Y_f_2,sigma_f_2=np.loadtxt(fname_O2,unpack=True)
except OSError:
    fname_O2=download_file('https://dcc.ligo.org/public/0158/T1900058/003/Figure_1_Cf_spectra_O2.dat')
    make_dir('data')
    fname_O2_new = 'data/{fname_O2}'.format(fname_O2=fname_O2)
    os.rename(fname_O2,fname_O2_new)
    freqs_2,Y_f_2,sigma_f_2=np.loadtxt(fname_O2_new,unpack=True)

var_f_2=sigma_f_2**2

In [ ]:
plt.plot(freqs_2,Y_f_2,color='blue',label='Y(f)')
plt.plot(freqs_2,sigma_f_2,color='black',label='+/-sigma(f)')
plt.plot(freqs_2,-sigma_f_2,color='black')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Omega')
plt.xlim(20,60)
plt.ylim(-1e-5,1e-5)
plt.legend()

In [ ]:
cal_O1_H1 = 4.8 / 100
cal_O1_L1 = 5.4 / 100
cal_O2_H1 = 2.6 / 100
cal_O2_L1 = 3.85 / 100

cal_O1= np.sqrt(cal_O1_H1**2 + cal_O1_L1**2)
cal_O2= np.sqrt(cal_O2_H1**2 + cal_O2_L1**2)

O1_data = Baseline('HL-O1',Y_f_1,var_f_1,freqs_1,calibration_epsilon=cal_O1)
O2_data = Baseline('HL-O2',Y_f_2,var_f_2,freqs_2,calibration_epsilon=cal_O2)

baselines=[O1_data,O2_data]

npoints=1024 # live points

# PE

In [ ]:
label = 'GWB_powerlaw'
outdir = 'outdir'

Amin,Amax=1e-13,1e-5
fref=25

O2_UL_a0=3.5e-8
O2_UL_a2_3=3.0e-8
O2_UL_a3=5.1e-9
O2_UL_aMarg=3.4e-8

likelihood = PowerLawGWBLikelihood(baselines,fref=fref)

## Marginalize over alpha with Gaussian prior

* 0 mean
* 3.5 standard deviation
* gives similar results to triangle prior

In [ ]:
cleanup_dir(outdir)

# parameters for alpha prior
alpha_mean=0
alpha_std=3.5

# define priors
priors = dict(A=bilby.core.prior.LogUniform(Amin,Amax, 'A'),
              alpha=bilby.core.prior.Gaussian(alpha_mean,alpha_std, 'alpha'))

# Run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=npoints,
    walks=10, outdir=outdir, label=label,maxmcmc=10000)
result.plot_corner()

In [ ]:
# extract samples

A=result.samples[:,0]
alpha=result.samples[:,1]

logA=np.log10(A)
results=np.transpose(np.array([logA,alpha]))

In [ ]:
# make a plot analogous to Fig. 5 of O2 isotropic paper (supplementary material) https://arxiv.org/abs/1903.02886?
corner.corner(results,smooth=0.5,labels=['log10 A','alpha'],levels=[0.68,0.95])
fig=plt.gcf()

ax1=fig.axes[0]
ax2=fig.axes[2]
ax3=fig.axes[3]

logAmin,logAmax=np.log10(Amin),np.log10(Amax)
ax1.set_xlim(logAmin,logAmax)
logAs = np.linspace(logAmin,logAmax)
y = len(logA) * np.ones(logAs.shape) / ((logAmax-logAmin)/np.log10(np.e))
ax1.plot(logAs,y,color='gray',linestyle='--')


ax2.set_xlim(-13,-5)
ax2.set_ylim(-8,8)
ax2.grid()


sig_a=3.5
ax3.set_xlim(-8,8)
alphas=np.linspace(-8,8,100)
y=len(logA)/np.sqrt(2*np.pi*alpha_std**2)*np.exp(-alphas**2/(2*alpha_std**2))
ax3.plot(alphas,y,color='gray',linestyle='--')

#UL=np.percentile(A,95)
#print('95%% UL = %e'%UL)
#print('95%% UL from O2 paper = %e'%O2_UL_aMarg)

plt.show()

In [ ]:
# extra plot: marginalized A posterior

x=np.linspace(np.log10(Amax),np.log10(Amin),10)
prior=np.ones(x.shape)/(np.log10(Amax)-np.log10(Amin))
plt.plot(x,prior,color='red',label='prior')

plt.hist(np.log10(A),bins=30,histtype='step',color='blue',density=True,label='posterior')
plt.axvline(np.percentile(np.log10(A),95),color='blue',linestyle='--')

plt.xlabel('log10A')
plt.ylabel('Probability density')

UL=np.percentile(A,95)
print('95%% UL = %e'%UL)
print('95%% UL from O2 paper = %e'%O2_UL_aMarg)

plt.legend()

In [ ]:
# extra plot: marginalized alpha posterior=

x=np.linspace(-10,10,1000)
prior=1/np.sqrt(2*np.pi*alpha_std**2) * np.exp(-0.5*(x/alpha_std)**2)
plt.plot(x,prior,color='red',label='prior')


plt.hist(alpha,bins=30,histtype='step',color='blue',density=True,label='posterior')
plt.axvline(np.percentile(alpha,5),color='blue',linestyle='--')
plt.axvline(np.percentile(alpha,95),color='blue',linestyle='--')


plt.xlabel('alpha')
plt.ylabel('Probability density')
plt.legend()

## alpha=0

In [ ]:
cleanup_dir(outdir)

alpha_fixed=0

priors = dict(A=bilby.core.prior.LogUniform(Amin,Amax, 'A'),
              alpha=bilby.core.prior.DeltaFunction(alpha_fixed, 'alpha'))

# Run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=npoints,
    walks=10, outdir=outdir, label=label,maxmcmc=10000)
result.plot_corner()

In [ ]:
A=result.samples[:,0]


x=np.linspace(np.log10(Amax),np.log10(Amin),10)
prior=np.ones(x.shape)/(np.log10(Amax)-np.log10(Amin))
plt.plot(x,prior,color='red',label='prior')

plt.hist(np.log10(A),bins=30,histtype='step',color='blue',density=True,label='posterior')
plt.axvline(np.percentile(np.log10(A),95),color='blue',linestyle='--')

plt.xlabel('log10A')
plt.ylabel('Probability density')

UL=np.percentile(A,95)
print('95%% UL = %e'%UL)
print('95%% UL from O2 paper = %e'%O2_UL_a0)
plt.legend()

## alpha=2/3

In [ ]:
cleanup_dir(outdir)

alpha_fixed=2./3

priors = dict(A=bilby.core.prior.LogUniform(Amin,Amax, 'A'),
              alpha=bilby.core.prior.DeltaFunction(alpha_fixed, 'alpha'))

# Run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=npoints,
    walks=10, outdir=outdir, label=label,maxmcmc=10000)
result.plot_corner()

In [ ]:
A=result.samples[:,0]

x=np.linspace(np.log10(Amax),np.log10(Amin),10)
prior=np.ones(x.shape)/(np.log10(Amax)-np.log10(Amin))
plt.plot(x,prior,color='red',label='prior')

plt.hist(np.log10(A),bins=30,histtype='step',color='blue',density=True,label='posterior')
plt.axvline(np.percentile(np.log10(A),95),color='blue',linestyle='--')

plt.xlabel('log10A')
plt.ylabel('Probability density')

UL=np.percentile(A,95)
print('95%% UL = %e'%UL)
print('95%% UL from O2 paper = %e'%O2_UL_a2_3)
plt.legend()

## alpha=3

In [ ]:
cleanup_dir(outdir)

alpha_fixed=3

priors = dict(A=bilby.core.prior.LogUniform(Amin,Amax, 'A'),
              alpha=bilby.core.prior.DeltaFunction(alpha_fixed, 'alpha'))

# Run sampler
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=npoints,
    walks=10, outdir=outdir, label=label,maxmcmc=10000)
result.plot_corner()

In [ ]:
A=result.samples[:,0]

x=np.linspace(np.log10(Amax),np.log10(Amin),10)
prior=np.ones(x.shape)/(np.log10(Amax)-np.log10(Amin))
plt.plot(x,prior,color='red',label='prior')

plt.hist(np.log10(A),bins=30,histtype='step',color='blue',density=True,label='posterior')
plt.axvline(np.percentile(np.log10(A),95),color='blue',linestyle='--')

plt.xlabel('log10A')
plt.ylabel('Probability density')

UL=np.percentile(A,95)
print('95%% UL = %e'%UL)
print('95%% UL from O2 paper = %e'%O2_UL_a3)
plt.legend()